In [1]:
 pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 6.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [5]:
from datasets import load_dataset
train_data = load_dataset (
    "mt_eng_vietnamese",
    "iwslt2015-en-vi",
    split="train"
)
valid_data = load_dataset (
     "mt_eng_vietnamese",
    "iwslt2015-en-vi",
    split="validation"
)
test_data = load_dataset (
    "mt_eng_vietnamese",
    "iwslt2015-en-vi",
    split="test"
)


Generating train split:   0%|          | 0/133318 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1269 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1269 [00:00<?, ? examples/s]

Dataset mt_eng_vietnamese downloaded and prepared to /root/.cache/huggingface/datasets/mt_eng_vietnamese/iwslt2015-en-vi/1.0.0/53add551a01e9874588066f89d42925f9fad43db347199dad00f7e4b0c905a71. Subsequent calls will reuse this data.


In [6]:
class DataPreparing:
  def __init__(self, save_data_dir , source_lang , target_lang):
    self.save_data_dir = save_data_dir
    self.source_lang = source_lang
    self.target_lang = target_lang
  def download_dataset(self):
    if not(os.path.exists(self.save_data_dir)):
      print('Create Foler')
      os.mkdir(self.save_data_dir)
    if len(os.listdir(self.save_data_dir)) ==0:
      print('#1-Download Dataset')
      corpus = datasets.load_dataset(  "mt_eng_vietnamese",
      "iwslt2015-en-vi",)
      print('#2-Save Dataset')
      for data in ['train' , 'validation' , 'test']:
        source_data, target_data = self.get_data(corpus[data])
        print('Source lang : {} - {} : {}'.format(self.source_lang, data , len(source_data)))
        print('Target lang : {} - {} : {}'.format(self.target_lang, data , len(target_data)))
        self.save_data(source_data, os.path.join(self.save_data_dir,data +  '.' + self.source_lang))
        self.save_data(target_data, os.path.join(self.save_data_dir,data + '.' + self.target_lang))
    else:
        print('Dataset exit!')
  def get_data(self,corpus):
    source_data = []
    target_data = []
    for data in corpus :
      source_data.append(data['translation'][self.source_lang])
      target_data.append(data['translation'][self.target_lang])
    return source_data, target_data
  def save_data(self,data,save_path):
    print('=> Save data => Path: {}'.format(save_path))
    with open(save_path,'w' , encoding='utf-8') as f:
      f.write('\n'.join(data))

In [7]:
def train_sentencepiece(cfg , is_src=True):
  template = "--input={} \
              --pad_id={} \
              --bos_id={} \
              --eos_id={} \
              --unk_id={} \
              --model_prefix={} \
              --vocab_size={} \
              --character_coverage={} \
              --model_type={}"
  if is_src:
    train_file = f"{cfg.data_dir}/train.{cfg.src_lang}"
    model_prefix = f"{cfg.sp_dir}/{cfg.src_model_prefix}"
  else:
     train_file = f"{cfg.data_dir}/train.{cfg.tgt_lang}"
     model_prefix = f"{cfg.sp_dir}/{cfg.tgt_model_prefix}"

  print(f"===> Processing file : {train_file}")
  if not os.path.isdir(cfg.sp_dir):
    os.mkdir(cfg.sp_dir)

  sp.cfg = template.format (
     train_file ,
     cfg.pad_id ,
     cfg.sos_id ,
     cfg.eos_id ,
     cfg.unk_id ,
     model_prefix ,
     cfg.sp_vocab_size ,
     cfg.character_coverage ,
     cfg.model_type                     )

  spn.SentencePieceTrainer.Train(sp_cfg)



In [8]:
from torch.utils.data import Dataset
class NMTDataset(Dataset) :
  def __init__(self , cfg  , data_type="train"):
    super().__init__()
    self.cfg = cfg

    self.sp_src , self.sp_tgt = self.load_sp_tokenizer()
    self.src_texts , self.tgt_texts = self.read_data(data_type)

    src_tokenized_sequences = self.texts_to_sequences(self.src_texts, True)
    tgt_input_tokenized_sequences , tgt_output_tokenized_sequences = self.texts_to_sequences(self.tgt_texts , False)

    self.src_data = torch.LongTensor(src_tokenized_sequences)
    self.input_tgt_data = torch.LongTensor(tgt_input_tokenized_sequences)
    self.output_tgt_data = torch.LongTensor(tgt_output_tokenized_sequences)

  def read_data(self , data_type) :
    print(f"===> Load data from : {self.cfg.data_dirr}/{data_type}.{self.cfg.src_lang}")
    with open(f"{self.cfg.data_dir}/{data_type}.{self.cfg.src_lang}" , 'r') as f:
        src_texts = f.readlines()
    print(f"===> Load data from: {self.cfg.data_dir}/{data_type}.{self.cfg.tgt_lang}")
    with open(f"{self.cfg.data_dir}/{data_type}.{self.cfg.tgt_lang}", 'r') as f:
        trg_texts = f.readlines()

    return src_texts , trg_texts
  def  load_sp_tokenizer(self):
    sp_src =spm.SentencePieceProcessor()
    sp_src.Load(f"{self.cfg.sp_dir}/{self.cfg.src_model_prefix}.model")

    sp_tgt = spm.SentencePieceProcessor()
    sp_tgt.Load(f"{self.cfg.sp_dir}/{self.cfg.tgt_model_prefix}.model")

    return sp_src ,sp_tgt
  def texts_to_sequences(self , texts , is_src = True) :
    if is_src :
      src_tokenized_sequences = []
      for text in tqdm(texts):
        tokenized = self.sp_src.EncodeAsIds(text.strip())
        src_tokenized_sequences.append(
            pad_or_truncate([self.cfg.sos_id]+tokenized + [self.cfg.eos_id] , self.cfg.seq_len , self.cfg.pad_id)
        )
      return src_tokenized_sequences
    else :
      tgt_input_tokenized_sequences = []
      tgt_output_tokenized_sequences = []
      for text in tqdm(texts):
        tokenized = self.sp_tgt.EncodeAsIds(text.strip())
        tgt_input = [self.cfg.sos_id] + tokenized
        tgt_output = tokenized +[self.cfg.eos_id]
        tgt_input_tokenized_sequences.append(pad_or_truncate(tgt_input , self.cfg.seq_len , self.cfg.pad_id))
        tgt_output_tokenized_sequences.append(pad_or_truncate(tgt_output , self.cfg.seq_len , self.cfg.pad_id))
      return tgt_input_tokenized_sequences , tgt_output_tokenized_sequences
  def __getitem__(self, idx) :
    return self.src_data[idx] , self.input_tgt_data[idx] , self.out.put_tgt_data[idx]
  def __len__(self):
    return np.shape(self.src_data) [0]
def pad_or_truncate(tokenized_sequence , seq_len , pad_id):
  if len(tokenized_sequence) < seq_len:
      left = seq_len - len(tokenized_sequence)
      padding = [pad_id] * left
      tokenized_sequence += padding
  else:
      tokenized_sequence = tokenized_sequence[:seq_len]
  return tokenized_sequence
def get_data_loader(cfg , data_type = 'train'):
  dataset = NMTDataset(cfg , data_type)

  if data_type == 'train' :
     shuffle = True
  else:
     shuffle = False
  dataloader = DataLoader(dataset , batch_size = cfg.batch_size , shuffle=shuffle)

  return dataset , dataloader


In [9]:
import torch
class BaseConfig :
  """ base Encoder Deconder config """
  def __init__(self, **kwargs):
    for k , v in kwargs.items():
      setattr(self , k , v)
class NMTConfig(BaseConfig):
  # Dataset
  data_dir = './data'
  src_lang = 'vi'
  tgt_lang = 'en'
  # Tokenizer
  sp_dir = data_dir + '/sp'
  pad_id =0
  sos_id =1
  eos_id =2
  unk_id =3
  src_model_prefix = 'sp_' + src_lang
  tgt_model_prefix = 'sp_' + tgt_lang
  sp_vocab_size = 10000
  character_coverage =1.0
  model_type ='unigram'

  # Training
  device =torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
  learning_rate =1e-4
  batch_size =256
  seq_len =150
  num_epochs =50
  ckpt_dir = './save_model'
  ckpt_name = 'best_ckpt.tar'
  beam_size = 3

In [10]:
from torch.utils.data import Dataset
def load_tokenizer(model_name_or_path):
  if ' bert' in model_name_or_path.split('-'):
    return BertTokenizerFast.from_pretrained(model_name_or_path)
  elif 'gpt2' in model_name_or_path.split('-'):
    return GPT2TokenizerFast.from_pretrained(model_name_or_path)
  else :
    return AutoTokenizer.from_pretrained(model_name_or_path)
def get_tokenizer(self) :
  if self.cfg.load_model_from_path :
    self.cfg.src_tokenizer = load_tokenizer(self.cfg.ckpt_dir)
    self.cfg.tgt_tokenizer = load_tokenizer(self.cfg.ckpt_dir)
  else :
    self.cfg.src_tokenizer = load_tokenizer(self.cfg.src_model_name)
    self.cfg.tgt_tokenizer = load_tokenizrt(self.cfg.tgt_model_name)
    if "bert" in self.cfg.tgt_model_name.split('-') :
       self.cfg.add_special_tokens = False
       self.cfg.bos_token_id = self.cfg.tgt_tokenizer.cls_token_id
       self.cfg.eos_token_id = self.cfg.tgt_tokenizer.sep_token_id
       self.cfg.pad_token_id = self.cfg.tgt_tokenizer.pad_token_id
    else :
      self.cfg.add_special_tokens= True
      self.cfg.tgt_tokenizer.add_special_tokens(
          {
              "bos_token": "[BOS]" ,
              "eos_token": "[EOS]" ,
              "pad_token": "[PAD]" ,
          }
      )
      self.cfg.bos_token_id = self.cfg.tgt_tokenizer.bos_token_id
      self.cfg.eos_token_id = self.cfg.tgt_tokenizer.eos_token_id
      self.cfg.pad_token_id = self.cfg.tgt_tokenizer.pad_token_id
      self.cfg.src_tokenizer.save_pretrained(
          os.path.join(self.cfg.ckpt_dir , f"{self.cfg.src_lang}_tokenizer_{cfg.src_model_name}")
      )
      self.cfg.tgt_tokenizer.save_pretrained(
          os.path.join(self.cfg.ckpt_dir,f"{self.cfg.tgt_lang}_tokenizer_{cfg.tgt_model_name}")
      )
class NMTDataset(Dataset):
  def __init__(self , cfg , data_type= "train"):
     super().__init__()
     self.cfg = cfg

     self.src_texts , self.tgt_texts = self.read_data(data_type)

     self.src_input_ids , self.src_attention_mask = self.texts_to_sequences(
        self.src_texts
     )
     self.tgt_input_ids , self.tgt_attention_mask , self.labels = self.texts_to_sequences(
        self.tgt_texts ,
        is_src= False
     )

  def read_data(self , data_type) :
    data = load_dataset(
        "mt_eng_vietnamese" ,
        "iwslt2015-en-vi" ,
        split = data_type
    )
    src_texts = [sample["translation"][self.cfg.src_lang] for sample in data]
    tgt_texts = [sample["translation"][self.cfg.tgt_lang] for sample in data]
    return src_texts , tgt_texts
  def texts_to_sequences(self , texts , is_src=True) :
    if is_src :
      src_inputs = self.cfg.src_tokenizer(
          texts ,
          padding = 'max_length' ,
          truncation=True ,
          max_length = self.cfg.src_max_len ,
          return_tensors='pt'
      )
      return (
          src_inputs.input_ids ,
          src_inputs.attention_mask
      )
    else :
      if self.cfg.add_special_tokens:
        texts = [
            ' '.join([
                self.cfg.tgt_tokenizer.bos_token ,
                text ,
                self.cfg.tgt_tokenizer.eos_token
            ])
            for text in texts
        ]
      tgt_inputs = self.cfg.tgt_tokenizer(
          texts ,
          padding = 'max_length' ,
          truncation = True ,
          max_length = self.cfg.tgt_max_len ,
          return_tensors = 'pt'
      )
      labels = tgt_inputs.input_ids.numpy().tolist()
      labels = [
          [
              -100 if token_id == self.cfg.tgt_tokenizer.pad_token_id else token_id
              for token_id in label
          ]
          for label in labels
      ]

      labels = torch.LongTensor(labels)

      return (
          tgt_inputs.input_ids ,
          tgt_inputs.attention_mask ,
          labels
      )

  def __getitem__(self , idx) :
    return {
        "input_ids" : self.src_input_ids[idx],
        "attention_mask": self.src_attention_mask[idx] ,
        "decoder_input_ids" : self.tgt_input_ids[idx] ,
        "decoder_attention_mask" : self.tgt_attention_mask[idx] ,
        "labels" : self.labels[idx]
    }
  def __len__(self):
    return np.shape(self.src_input_ids)[0]


In [11]:
def get_model(self):
  if self.cfg.load_model_from_path:
    save_model_path = os.path.join(self.cfg.ckpt_dir , self.cfg.ckpt_name)
    self.model = EncoderDecoderModel.from_pretrained(save_model_path)
  else :
    self.model = EncoderDecoderModel.from_encoder_decoder_pretrained(
        self.cfg.src_model_name ,
        self.cfg.tgt_model_name
    )
    self.model.decoder.resize_token_embeddings(len(self.cfg.tgt_tokenizer))
    self.model.config.decoder_start_token_id = self.cfg.bos_token_id
    self.model.config.eos_token_id = self.cfg.eos_token_id
    self.model.config.pad_token_id = self.cfg.pad_token_id
    self.model.config.vocab_size = len(self.cfg.tgt_tokenizer)
    self.model.config.max_length = self.cfg.max_length_decoder
    self.model.config.min_length = self.cfg.min_length_decoder
    self.model.config.co_repeat_ngram_size = 3
    self.model.config.earlu_stopping = True
    self.model.config.length_penalty =2.0
    self.model.config.num_beams = self.cfg.beam_size

In [12]:
class Manager():
  def __init__(self , cfg , is_train = True) :
    self.cfg = cfg

    print("Loading TOkenizrt ... ")
    self.get_tokenizer()

    print("Loading Model... ")
    self.get_model()

    print("Loading Metric ... ")
    self.bleu_metric = load_metric("sacrebleu")

    print("Check Save Model Path")
    if not os.path.exists(self.cfg.ckpt_dir):
      os.mkdir(self.cfg.ckpt_dir)

    if is_train:
       # Load dataloaders
       print("Loading Dataset...")
       self.train_dataset = NMTDataset(self.cfg , data_type = "train")
       self.valid_dataset = NMTDataset(self.cfg , data_type = "validation")
    print("Setting finished .")
  def get_tokenizer(self) :
    if self.cfg.load_model_from_path :
      self.cfg.src_tokenizer = load_tokenizer(self.cfg.ckpt_dir)
      self.cfg.tgt_tokenizer = load_tokenizer(self.cfg.ckpt_dir)
    else :
      self.cfg.src_tokenizer = load_tokenizer(self.cfg.src_model_name)
      self.cfg.tgt_tokenizer = load_tokenizer(self.cfg.tgt_model_name)
      if "bert" in self.cfg.tgt_model_name.split('-') :
        self.cfg.add_special_tokens = False
        self.cfg.bos_token_id = self.cfg.tgt_tokenizer.cls_token_id
        self.cfg.eos_token_id = self.cfg.tgt_tokenizer.sep_token_id
        self.cfg.pad_token_id = self.cfg.tgt_tokenizer.pad_token_id
      else :
        self.cfg.add_special_tokens = True
        self.cfg.tgt_tokenizer.add_special_tokens(
            {
                "bos_token" : "[BOS]" ,
                "eos_token" : "[EOS]" ,
                "pad_token" : "[PAD]"
            }
        )
        self.cfg.bos_token_id = self.cfg.tgt_tokenizer.bos_token_id
        self.cfg.eos_token_id = self.cfg.tgt_tokenizer.eos_token_id
        self.cfg.pad_token_id = self.cfg.tgt_tokenizer.pad_token_id
    self.cfg.src_tokenizer.save_pretrained(
        os.path.join(self.cfg.ckpt_dir , f"{self.cfg.src_lang}_tokenizer_{cfg.src_model_name}")
    )
    self.cfg.tgt_tokenizer.save_pretrained(
        os.path.join(self.cfg.ckpt_dir, f"{self.cfg.tgt_lang}_tokenizer_{cfg.tgt_model_name}")
    )
  def get_model(self) :
        if self.cfg.load_model_from_path:
         save_model_path = os.path.join(self.cfg.ckpt_dir , self.cfg.ckpt_name)
         self.model = EncoderDecoderModel.from_pretrained(save_model_path)
        else:
         self.model = EncoderDecoderModel.from_encoder_decoder_pretrained(
            self.cfg.src_model_name ,
            self.cfg.tgt_model_name
         )
         self.model.decoder.resize_token_embeddings(len(self.cfg.tgt_tokenizer))
         self.model.config.decoder_start_token_id = self.cfg.bos_token_id
         self.model.config.eos_token_id = self.cfg.eos_token_id
         self.model.config.pad_token_id = self.cfg.pad_token_id
         self.model.config.vocab_size = len(self.cfg.tgt_tokenizer)
         self.model.config.max_length = self.cfg.max_length_decoder
         self.model.config.min_length = self.cfg.min_length_decoder
         self.model.config.no_repeat_ngram_size = 3
         self.model.config.early_stopping = True
         self.model.config.length_penalty = 2.0
         self.model.config.num_beams = self.cfg.beam_size

  def train(self) :
    print("Training...")
    if self.cfg.use_eval_steps:
        training_args = Seq2SeqTrainingArguments(
            predict_with_generate = True ,
            evaluation_strategy = "steps" ,
            save_strategy = 'steps' ,
            save_steps = self.cfg.eval_steps ,
            eval_steps = self.cfg.eval_steps ,
            output_dir = self.cfg.ckpt_dir ,
            per_device_train_batch_size = self.cfg.train_batch_size ,
            per_device_eval_batch_size = self.cfg.eval_batch_size ,
            learning_rate = self.cfg.learning_rate ,
            weight_decay = 0.005 ,
            num_train_epochs = self.cfg.num_train_epochs

        )
    else :
        training_args = Seq2SeqTrainingArguments(
            predict_with_generate = True ,
            evaluation_strategy = "epoch" ,
            save_strategy = 'epoch' ,
            output_dir = self.cfg.ckpt_dir ,
            per_device_train_batch_size = self.cfg.train_batch_size ,
            per_device_eval_batch_size = self.cfg.eval_batch_size ,
            learning_rate = self.cfg.learning_rate ,
            weight_decay = 0.005 ,
            num_train_epochs = self.cfg.num_train_epochs
        )
    data_collator = DataCollatorForSeq2Seq(
        self.cfg.tgt_tokenizer ,
        model=self.model
    )
    trainer = Seq2SeqTrainer(
        self.model ,
        training_args ,
        train_dataset = self.train_dataset ,
        eval_dataset = self.valid_dataset ,
        data_collator = data_collator ,
        tokenizer = self.cfg.tgt_tokenizer ,
        compute_metrics = self.compute_metrics
    )

    trainer.train()
  def  compute_metrics(self , eval_preds):
    preds , labels = eval_preds
    if isinstance(preds , tuple):
      preds = preds[0]
    decoded_preds =self.cfg.tgt_tokenizer.batch_decode(preds , skip_special_tokens =True)
    labels = np.where(labels != -100 , labels , self.cfg.tgt_tokenizer.pad_token_id)
    decoded_labels = self.cfg.tgt_tokenizer.batch_decode(labels , skip_special_tokens = True)
    decoded_preds ,decoded_labels = postprocess_text(decoded_preds , decoded_labels)
    result =self.bleu_metric.compute(
        predictions = decoded_preds ,
        references = decoded_labels
    )
    resul = {"bleu_score" : result["score"]}

    prediction_lens = [np.count_nonzero(pred != self.cfg.tgt_tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = { k : round(v , 4) for k ,v in result.items()}

    return result
class BaseConfig :
  """ base Encoder Decoder config """
  def __init__(self , **kwargs):
    for k , v in kwargs.items():
      setattr(self , k , v)
class NMTConfig(BaseConfig):
  # Data
  src_lang = 'vi'
  tgt_lang = 'en'
  src_max_len = 75
  tgt_max_len = 75
  # model
  src_model_name  = "bert-base-multilingual-cased"
  tgt_model_name = "bert-base-multilingual-cased"
  # Trainign
  load_model_from_path = False
  device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
  learning_rate = 3e-5
  train_batch_size = 32
  eval_batch_size = 32
  num_train_epochs = 10
  ckpt_dir = src_model_name + '_to_' + tgt_model_name
  use_eval_steps = False
  eval_steps = 2000

  # Inference
  max_length_decoder = 75
  min_length_decoder = 25
  beam_size = 5

cfg = NMTConfig()





In [13]:
!pip install -U accelerate
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.20.3
    Uninstalling accelerate-0.20.3:
      Successfully uninstalled accelerate-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 73.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.2
    Uninstalling transformers-4.30.2:
      Successfully uninstalled transformers-4.30.2


In [14]:
import os
from transformers import AutoTokenizer
from transformers import EncoderDecoderModel
from transformers import Seq2SeqTrainingArguments
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer
import numpy as np
from datasets import load_metric
manager = Manager(cfg)
manager.train()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

Loading TOkenizrt ... 


Loading Model... 


Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['bert.encoder.layer.7.crossattention.self.value.weight', 'bert.encoder.layer.6.crossattention.self.value.weight', 'bert.encoder.layer.6.crossattention.self.query.weight', 'bert.encoder.layer.9.crossattention.self.key.weight', 'bert.encoder.layer.2.crossattention.output.dense.bias', 'bert.encoder.layer.1.crossattention.self.value.weight', 'bert.encoder.layer.5.crossattention.self.key.weight', 'bert.encoder.layer.2.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.8.crossattention.output.dense.weight', 'bert.encoder.layer.9.crossattention.output.dense.bias', 'bert.encoder.layer.2.crossattention.self.query.weight', 'bert.encoder.layer.1.crossattention.self.key.bias', 'bert.encoder.layer.3.crossattention.self.value.bias', 'bert.encoder.layer.3.crossattention.output.LayerNorm.weight', 'bert.encode

Loading Metric ... 


Check Save Model Path
Loading Dataset...
Setting finished .
Training...


UsageError: api_key not configured (no-tty). call wandb.login(key=[your_api_key])